### Tarea #5
#### Juan Pablo Fernandez Delgado
#### Big Data

Creación de la sesión Spark:

In [1]:
#import SparkSession
from pyspark.sql import SparkSession

In [2]:
# Crear el spark session object, llamarle "supervised_ml"
spark=SparkSession.builder.appName('supervised_ml').getOrCreate()

## Regression 

Carga de datos, archivo *Linear_regression_dataset.csv*:

In [4]:
# Carga de datos

df=spark.read.csv('Linear_regression_dataset.csv',inferSchema=True,header=True)

Se invocan las librerias correcpondientes a **LinearRegression**, asi como las de OneHotEncoder, StringIndexer, VectorAssembler:

In [5]:
# Importacion de libs y operaciones

from pyspark.ml.feature import OneHotEncoder, StringIndexer, VectorAssembler
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator

Se visualizan algunos datos:

In [6]:
print((df.count(), len(df.columns))) 

(1232, 6)


Se muestran los primeros 10 datos:

In [7]:
# primeros 10 datos
df.show(10)

+-----+-----+-----+-----+-----+-----+
|var_1|var_2|var_3|var_4|var_5|label|
+-----+-----+-----+-----+-----+-----+
|  734|  688|   81|0.328|0.259|0.418|
|  700|  600|   94| 0.32|0.247|0.389|
|  712|  705|   93|0.311|0.247|0.417|
|  734|  806|   69|0.315| 0.26|0.415|
|  613|  759|   61|0.302| 0.24|0.378|
|  748|  676|   85|0.318|0.255|0.422|
|  669|  588|   97|0.315|0.251|0.411|
|  667|  845|   68|0.324|0.251|0.381|
|  758|  890|   64| 0.33|0.274|0.436|
|  726|  670|   88|0.335|0.268|0.422|
+-----+-----+-----+-----+-----+-----+
only showing top 10 rows



## Feature Engineering

Creamos un solo vector con todos los features i.e 'var_1', 'var_2', 'var_3', 'var_4', 'var_5', a este le llamaremos "features" y como salida colocamos a 'label':

In [8]:
# Vector Ensamblador

df_assembler =  VectorAssembler(inputCols=['var_1', 'var_2', 'var_3', 'var_4', 'var_5'], outputCol="features")
df = df_assembler.transform(df) 

In [9]:
# visulizacion de vector ensamblado compuesto por features y label
df.select(['features','label']).show(20,False)

+------------------------------+-----+
|features                      |label|
+------------------------------+-----+
|[734.0,688.0,81.0,0.328,0.259]|0.418|
|[700.0,600.0,94.0,0.32,0.247] |0.389|
|[712.0,705.0,93.0,0.311,0.247]|0.417|
|[734.0,806.0,69.0,0.315,0.26] |0.415|
|[613.0,759.0,61.0,0.302,0.24] |0.378|
|[748.0,676.0,85.0,0.318,0.255]|0.422|
|[669.0,588.0,97.0,0.315,0.251]|0.411|
|[667.0,845.0,68.0,0.324,0.251]|0.381|
|[758.0,890.0,64.0,0.33,0.274] |0.436|
|[726.0,670.0,88.0,0.335,0.268]|0.422|
|[583.0,794.0,55.0,0.302,0.236]|0.371|
|[676.0,746.0,72.0,0.317,0.265]|0.4  |
|[767.0,699.0,89.0,0.332,0.274]|0.433|
|[637.0,597.0,86.0,0.317,0.252]|0.374|
|[609.0,724.0,69.0,0.308,0.244]|0.382|
|[776.0,733.0,83.0,0.325,0.259]|0.437|
|[701.0,832.0,66.0,0.325,0.26] |0.39 |
|[650.0,709.0,74.0,0.316,0.249]|0.386|
|[804.0,668.0,95.0,0.337,0.265]|0.453|
|[713.0,614.0,94.0,0.31,0.238] |0.404|
+------------------------------+-----+
only showing top 20 rows



Partimos a continuación el set de datos en 75% training y 25% testing:

In [12]:
# Particion del data set

model_df=df.select(['features','label'])
train,test = model_df.randomSplit([0.75,0.25])

print(f"Size of train Dataset : {train.count()}" )
print(f"Size of test Dataset : {test.count()}" )

Size of train Dataset : 921
Size of test Dataset : 311


Creamos el Regresor Lineal: 

In [13]:
lr = LinearRegression()

Entrenamos el modelo de regresión lineal:

In [14]:
# Fit the model, le llamamos lr_model

lr_model = lr.fit(train)

Creamos el dataframe de prediciones (*predictions_df*) a partir del modelo de entrenamiento y el conjunto de datos test: 

In [15]:
predictions_df = lr_model.transform(test)

Visualizamos el contenido de *predictions_df*:

In [16]:
# visulizacion de predictions_df
predictions_df.show(10)


+--------------------+-----+-------------------+
|            features|label|         prediction|
+--------------------+-----+-------------------+
|[473.0,499.0,73.0...|0.315|0.31621302513413063|
|[498.0,615.0,67.0...|0.318|0.32281323041385945|
|[522.0,621.0,72.0...|0.317| 0.3281207707448808|
|[528.0,652.0,71.0...|0.319|0.33151438214244877|
|[531.0,734.0,55.0...| 0.34|0.34226926418514714|
|[536.0,531.0,83.0...|0.318| 0.3274531740237425|
|[548.0,636.0,63.0...|0.339| 0.3351731663739578|
|[558.0,688.0,67.0...| 0.35|0.34579183723554857|
|[571.0,577.0,83.0...|0.368| 0.3564831626706276|
|[573.0,656.0,75.0...|0.345|0.34497948688670066|
+--------------------+-----+-------------------+
only showing top 10 rows



Ahora, evaluamos el modelo de Regresión Lineal, con los datos de TEST:

In [27]:
# evaluacion del modelo, le llamaremos model_predictions

model_predictions = lr_model.evaluate(test)

Imprimimos el valor de R2:

In [28]:
# valor de R2
lr_evaluator = RegressionEvaluator(metricName='r2')
lr_r2 = lr_evaluator.evaluate(predictions_df)

print(f'Regression Lineal is {lr_r2}')

Regression Lineal is 0.869737580749536


Imprimimos el valor del meanSquaredError:

In [29]:
# valor del meanSquaredError

lr_rmse = lr_evaluator.evaluate(predictions_df)
print(f'RMSE {lr_rmse}')

RMSE 0.869737580749536


## Regresión con Árboles de Decisión

Importamos la librería *DecisionTreeRegressor*: 

In [30]:
# import lib
from pyspark.ml.regression import DecisionTreeRegressor

Creamos el Regresor DT, le llamaremos *dec_tree*:

In [31]:
# dec_tree

dec_tree = DecisionTreeRegressor()

Entrenamos el modelo:

In [32]:
# Train model, le llamaremos dec_tree_model
dec_tree_model = dec_tree.fit(train)

Cuánto es la profundidad máxima por defecto, de este algoritmo?

R/ 

Desplegamos las *featureImportances*:

In [33]:
dec_tree_model.featureImportances

SparseVector(5, {0: 0.966, 1: 0.0163, 2: 0.0003, 3: 0.006, 4: 0.0115})

Evaluamos el modelo con los datos de entrenamiento:

In [34]:
# Make predictions, le llamaremos model_predictions 

model_predictions = dec_tree_model.transform(test)

In [35]:
# visualizamos

model_predictions.show()

+--------------------+-----+-------------------+
|            features|label|         prediction|
+--------------------+-----+-------------------+
|[473.0,499.0,73.0...|0.315|0.32010000000000005|
|[498.0,615.0,67.0...|0.318|0.32010000000000005|
|[522.0,621.0,72.0...|0.317|0.32010000000000005|
|[528.0,652.0,71.0...|0.319|0.32010000000000005|
|[531.0,734.0,55.0...| 0.34|             0.3345|
|[536.0,531.0,83.0...|0.318|0.32010000000000005|
|[548.0,636.0,63.0...|0.339|0.34600000000000003|
|[558.0,688.0,67.0...| 0.35| 0.3461794871794871|
|[571.0,577.0,83.0...|0.368| 0.3461794871794871|
|[573.0,656.0,75.0...|0.345| 0.3461794871794871|
|[574.0,586.0,81.0...| 0.36| 0.3461794871794871|
|[575.0,680.0,68.0...|0.344| 0.3461794871794871|
|[575.0,864.0,55.0...|0.379| 0.3563571428571429|
|[576.0,759.0,57.0...| 0.35| 0.3729090909090911|
|[578.0,633.0,76.0...|0.337| 0.3461794871794871|
|[578.0,812.0,56.0...|0.351| 0.3563571428571429|
|[581.0,724.0,64.0...|0.346| 0.3563571428571429|
|[583.0,532.0,80.0..

Importamos el **RegressionEvaluator**

In [36]:
# import Evaluator
from pyspark.ml.evaluation import RegressionEvaluator

Usando *RegressionEvaluator* calculamos e imprimimos el valor de las metricas R2 y RMSE:

In [37]:
# R2 value of the model on test data 
dt_evaluator = RegressionEvaluator(metricName='r2')
dt_r2 = dt_evaluator.evaluate(model_predictions)
print(f'The r-square value of DecisionTreeRegressor is {dt_r2}')

# RMSE value of the model on test data 

rmse = dt_evaluator.evaluate(model_predictions)
print(f'RMSE {rmse}')

The r-square value of DecisionTreeRegressor is 0.8267324878389362
RMSE 0.8267324878389362


## RandomForestRegressor

Importamos a *RandomForestRegressor*

In [38]:
# import lib

from pyspark.ml.regression import RandomForestRegressor

Creamos el Regresor RF:

In [39]:
# Regresor 
rf = RandomForestRegressor() 

Entrenamos el modelo:

In [40]:
# Train model, le llamaremos rf_model
rf_model= rf.fit(train)

Desplegamos las *featureImportances*:

In [41]:
# importances 
rf_model.featureImportances

SparseVector(5, {0: 0.5634, 1: 0.0247, 2: 0.0121, 3: 0.2152, 4: 0.1846})

Desplegamos el numero de arboles (Num of Trees)

In [46]:
# Numero de Trees

rf_model.numTrees

Param(parent='RandomForestRegressor_c5954b45be98', name='numTrees', doc='Number of trees to train (>= 1).')

Evaluamos el modelo con los datos de entrenamiento, le llamaremos model_predictions:


In [47]:
# model_predictions
model_predictions = rf_model.transform(test)

Desplegamos los valores del *model_predictions*

In [48]:
model_predictions.show()

+--------------------+-----+-------------------+
|            features|label|         prediction|
+--------------------+-----+-------------------+
|[473.0,499.0,73.0...|0.315| 0.3282402798247504|
|[498.0,615.0,67.0...|0.318| 0.3255263839428545|
|[522.0,621.0,72.0...|0.317|0.32855320717517766|
|[528.0,652.0,71.0...|0.319|0.33495125462578396|
|[531.0,734.0,55.0...| 0.34|0.33930937910143155|
|[536.0,531.0,83.0...|0.318|0.32618277982475047|
|[548.0,636.0,63.0...|0.339|  0.347588661361918|
|[558.0,688.0,67.0...| 0.35| 0.3467159463332143|
|[571.0,577.0,83.0...|0.368|0.35140024856983787|
|[573.0,656.0,75.0...|0.345| 0.3489012138517681|
|[574.0,586.0,81.0...| 0.36| 0.3479949958525711|
|[575.0,680.0,68.0...|0.344|0.35000184035011384|
|[575.0,864.0,55.0...|0.379|0.35733106364920186|
|[576.0,759.0,57.0...| 0.35|0.36242669932201305|
|[578.0,633.0,76.0...|0.337|  0.351896348163144|
|[578.0,812.0,56.0...|0.351| 0.3446028801155832|
|[581.0,724.0,64.0...|0.346|0.35586798918139717|
|[583.0,532.0,80.0..

Usando *RegressionEvaluator* calculamos e imprimimos el valor de las metricas R2 y RMSE:

In [50]:
# R2 value of the model on test data 

df_evaluator = RegressionEvaluator(metricName='r2')
df_r2 = dt_evaluator.evaluate(model_predictions)
print(f'The r-square value is {df_r2}')

# RMSE value of the model on test data 

rmse = df_evaluator.evaluate(model_predictions)
print(f'RMSE {rmse}')


The r-square value is 0.8556265614934404
RMSE 0.8556265614934404


## Gradient-Boosted Tree Regressor

Importamos a GBTRegressor


In [52]:
# import
from pyspark.ml.regression import GBTRegressor

Creamos el Regresor GBTR, le llamaremos gbt:


In [53]:
# regresor
gbt = GBTRegressor()

Entrenamos el modelo:

In [54]:
# Train model, le llamaremos gbt_model
gbt_model = gbt.fit(train)

Desplegamos las featureImportances:

In [55]:
#Importances
gbt_model.featureImportances

SparseVector(5, {0: 0.599, 1: 0.1176, 2: 0.0927, 3: 0.0969, 4: 0.0939})

Evaluamos el modelo con los datos de entrenamiento, le llamaremos model_predictions:

In [56]:
# Model
model_predictions = gbt_model.transform(test)

Desplegamos los valores del *model_predictions*

In [57]:
# show 
model_predictions.show()

+--------------------+-----+-------------------+
|            features|label|         prediction|
+--------------------+-----+-------------------+
|[473.0,499.0,73.0...|0.315|  0.320645097047627|
|[498.0,615.0,67.0...|0.318|0.32100161305632124|
|[522.0,621.0,72.0...|0.317| 0.3155993442255766|
|[528.0,652.0,71.0...|0.319| 0.3230075830107691|
|[531.0,734.0,55.0...| 0.34| 0.3332814350401513|
|[536.0,531.0,83.0...|0.318| 0.3186486711478113|
|[548.0,636.0,63.0...|0.339|0.34391676944140537|
|[558.0,688.0,67.0...| 0.35|0.34434085701923595|
|[571.0,577.0,83.0...|0.368|0.35125559458403294|
|[573.0,656.0,75.0...|0.345| 0.3396907238756532|
|[574.0,586.0,81.0...| 0.36|0.35150262169602803|
|[575.0,680.0,68.0...|0.344| 0.3465959452916396|
|[575.0,864.0,55.0...|0.379|0.36249149128251906|
|[576.0,759.0,57.0...| 0.35| 0.3744222750558752|
|[578.0,633.0,76.0...|0.337|0.34444695935584474|
|[578.0,812.0,56.0...|0.351|0.35245322470900015|
|[581.0,724.0,64.0...|0.346|0.35356796895811904|
|[583.0,532.0,80.0..

Usando RegressionEvaluator calculamos e imprimimos el valor de las metricas R2 y RMSE:

In [59]:
 #Select (prediction, true label) and compute test error
# R2 value of the model on test data 

gbt_evaluator = RegressionEvaluator(metricName='r2')
gbt_r2 = dt_evaluator.evaluate(model_predictions)
print(f'The r-square value is {gbt_r2}')

# RMSE value of the model on test data 
rmse = gbt_evaluator.evaluate(model_predictions)
print(f'RMSE {rmse}')


The r-square value is 0.8198389869085505
RMSE 0.8198389869085505


 ## Exploracion de datos...

Usaremos el dataset https://archive.ics.uci.edu/ml/datasets/Bank+Marketing 

Indique a grandes razgos de que se trata este dataset:


Carga de datos, archivo bank_data.csv:


In [60]:
# Load csv Dataset 
df=spark.read.csv('bank_data.csv',inferSchema=True,header=True)

Determine la cantidad de datos en el dataset:

In [61]:
#number of records
print((df.count(),len(df.columns)))

(41188, 21)


A que dato corresponde cada columna?

In [62]:
# columns values
df.columns

['age',
 'job',
 'marital',
 'education',
 'default',
 'housing',
 'loan',
 'contact',
 'month',
 'day_of_week',
 'duration',
 'campaign',
 'pdays',
 'previous',
 'poutcome',
 'emp.var.rate',
 'cons.price.idx',
 'cons.conf.idx',
 'euribor3m',
 'nr.employed',
 'target_class']

Imprima el Schema:

In [63]:
#dataype of input data - Schema
df.printSchema()

root
 |-- age: integer (nullable = true)
 |-- job: string (nullable = true)
 |-- marital: string (nullable = true)
 |-- education: string (nullable = true)
 |-- default: string (nullable = true)
 |-- housing: string (nullable = true)
 |-- loan: string (nullable = true)
 |-- contact: string (nullable = true)
 |-- month: string (nullable = true)
 |-- day_of_week: string (nullable = true)
 |-- duration: integer (nullable = true)
 |-- campaign: integer (nullable = true)
 |-- pdays: integer (nullable = true)
 |-- previous: integer (nullable = true)
 |-- poutcome: string (nullable = true)
 |-- emp.var.rate: double (nullable = true)
 |-- cons.price.idx: double (nullable = true)
 |-- cons.conf.idx: double (nullable = true)
 |-- euribor3m: double (nullable = true)
 |-- nr.employed: double (nullable = true)
 |-- target_class: string (nullable = true)



En cuanto a la salida, como es la distrubución de clases?

In [66]:
# YES/NO Class Distribution

df.groupBy('target_class').count().show()



+------------+-----+
|target_class|count|
+------------+-----+
|          no|36548|
|         yes| 4640|
+------------+-----+



Una tarea típica, resulta de convertir los valores binarios en 1 y 0, usando como referencia "label", convierta los no/yes en 0/1:

In [67]:
from pyspark.sql import functions as F
from pyspark.sql import *
from functools import reduce

In [69]:

cols = ['target_class']

In [70]:
# New 1/0 Class Distribution
reduce(lambda df, c: df.withColumn(c, F.when(df[c] == 'no', 0).otherwise(1)), cols, df).show()


+---+-----------+--------+-------------------+-------+-------+----+---------+-----+-----------+--------+--------+-----+--------+-----------+------------+--------------+-------------+---------+-----------+------------+
|age|        job| marital|          education|default|housing|loan|  contact|month|day_of_week|duration|campaign|pdays|previous|   poutcome|emp.var.rate|cons.price.idx|cons.conf.idx|euribor3m|nr.employed|target_class|
+---+-----------+--------+-------------------+-------+-------+----+---------+-----+-----------+--------+--------+-----+--------+-----------+------------+--------------+-------------+---------+-----------+------------+
| 56|  housemaid| married|           basic.4y|     no|     no|  no|telephone|  may|        mon|     261|       1|  999|       0|nonexistent|         1.1|        93.994|        -36.4|    4.857|     5191.0|           0|
| 57|   services| married|        high.school|unknown|     no|  no|telephone|  may|        mon|     149|       1|  999|       0|

A continuación se presenta un ejercicio de Deep Learning para su revisión...

# Deep Learning 

Importamos las librerias necesarias:

In [71]:
import os
import numpy as np
import pandas as pd
from pyspark.sql.types import *
from pyspark.ml import Pipeline
from pyspark.sql import functions as f
from pyspark.sql.functions import udf, StringType
from pyspark.sql import SparkSession, functions as F
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.classification import MultilayerPerceptronClassifier
from pyspark.ml.feature import OneHotEncoder, VectorAssembler, StringIndexer

Inicializamos la sesion SPARK:

In [72]:
spark = SparkSession.builder.appName('deep_learning').getOrCreate()

Leemos el dataset:

In [73]:
data = spark.read.csv('dl_data.csv', header=True, inferSchema=True)

In [74]:
data.printSchema()

root
 |-- Visit_Number_Bucket: string (nullable = true)
 |-- Page_Views_Normalized: double (nullable = true)
 |-- Orders_Normalized: integer (nullable = true)
 |-- Internal_Search_Successful_Normalized: double (nullable = true)
 |-- Internal_Search_Null_Normalized: double (nullable = true)
 |-- Email_Signup_Normalized: double (nullable = true)
 |-- Total_Seconds_Spent_Normalized: double (nullable = true)
 |-- Store_Locator_Search_Normalized: double (nullable = true)
 |-- Mapped_Last_Touch_Channel: string (nullable = true)
 |-- Mapped_Mobile_Device_Type: string (nullable = true)
 |-- Mapped_Browser_Type: string (nullable = true)
 |-- Mapped_Entry_Pages: string (nullable = true)
 |-- Mapped_Site_Section: string (nullable = true)
 |-- Mapped_Promo_Code: string (nullable = true)
 |-- Maped_Product_Name: string (nullable = true)
 |-- Mapped_Search_Term: string (nullable = true)
 |-- Mapped_Product_Collection: string (nullable = true)



Renombramos la columna TARGET:

In [75]:
data = data.withColumnRenamed('Orders_Normalized', 'label')

In [76]:
data.printSchema()

root
 |-- Visit_Number_Bucket: string (nullable = true)
 |-- Page_Views_Normalized: double (nullable = true)
 |-- label: integer (nullable = true)
 |-- Internal_Search_Successful_Normalized: double (nullable = true)
 |-- Internal_Search_Null_Normalized: double (nullable = true)
 |-- Email_Signup_Normalized: double (nullable = true)
 |-- Total_Seconds_Spent_Normalized: double (nullable = true)
 |-- Store_Locator_Search_Normalized: double (nullable = true)
 |-- Mapped_Last_Touch_Channel: string (nullable = true)
 |-- Mapped_Mobile_Device_Type: string (nullable = true)
 |-- Mapped_Browser_Type: string (nullable = true)
 |-- Mapped_Entry_Pages: string (nullable = true)
 |-- Mapped_Site_Section: string (nullable = true)
 |-- Mapped_Promo_Code: string (nullable = true)
 |-- Maped_Product_Name: string (nullable = true)
 |-- Mapped_Search_Term: string (nullable = true)
 |-- Mapped_Product_Collection: string (nullable = true)



Partimos lo datos en Train, Validation y Test:

In [85]:
train, validation, test  = data.randomSplit([0.7, 0.2, 0.1], 1234)

Construimos el Pipeline

In [86]:
categorical_columns = [item[0] for item in data.dtypes if item[1].startswith('string')]
numeric_columns = [item[0] for item in data.dtypes if item[1].startswith('double')]

indexers = [StringIndexer(inputCol=column, outputCol='{0}_index'.format(column)) for column in categorical_columns]

featuresCreator = VectorAssembler(inputCols=[indexer.getOutputCol() for indexer in indexers] + numeric_columns, outputCol="features")

layers = [len(featuresCreator.getInputCols()), 4, 2, 2]

classifier = MultilayerPerceptronClassifier(labelCol='label', featuresCol='features', maxIter=100, layers=layers, blockSize=128, seed=1234)

pipeline = Pipeline(stages=indexers + [featuresCreator, classifier])

Entrenamos...

In [88]:
model = pipeline.fit(train)

Validamos y Evaluamos

In [89]:
train_output_df = model.transform(train)
validation_output_df = model.transform(validation)
test_output_df = model.transform(test)

Llevamos a cabo, algunas predicciones:

In [90]:
train_predictionAndLabels = train_output_df.select("prediction", "label")
validation_predictionAndLabels = validation_output_df.select("prediction", "label")
test_predictionAndLabels = test_output_df.select("prediction", "label")

metrics = ['weightedPrecision', 'weightedRecall', 'accuracy']

for metric in metrics:
    evaluator = MulticlassClassificationEvaluator(metricName=metric)
    print('Train ' + metric + ' = ' + str(evaluator.evaluate(train_predictionAndLabels)))
    print('Validation ' + metric + ' = ' + str(evaluator.evaluate(validation_predictionAndLabels)))
    print('Test ' + metric + ' = ' + str(evaluator.evaluate(test_predictionAndLabels)))

Train weightedPrecision = 0.9678672585062094
Validation weightedPrecision = 0.9691383210897103
Test weightedPrecision = 0.9694943540959478
Train weightedRecall = 0.967375637879319
Validation weightedRecall = 0.9685636856368564
Test weightedRecall = 0.9691266079891672
Train accuracy = 0.9673756378793191
Validation accuracy = 0.9685636856368564
Test accuracy = 0.9691266079891673


Puede mejorar el test accuracy del modelo variando alguno de los hyperparametros?